https://github.com/pytorch/examples/blob/a38cbfc6f817d9015fc67a6309d3a0be9ff94ab6/mnist/main.py

In [8]:
%%bash
pip install torch

In [9]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [11]:
log_interval=40

def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [12]:


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [13]:
device = torch.device("cpu")
batch_size=64
test_batch_size=1000
epochs=2
lr=1.0
gamma=0.7

In [14]:
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.285568
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.400332
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.353395
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.163749
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.435301
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.126560
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.160683
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.083055
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.028808
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.230672
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.103924
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.120122
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.110109
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.106920
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.145347
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.133720
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.150530
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.146240
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.117328
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.109137
Train Epoc

KeyboardInterrupt: 